# Fama French 3-Factor Model
By Robert Yip

Fama French 3-factor model built from the Mstar CPMS Aggressive and Conservative Canadian strategies.  
Takes annual snapshots of the portfolios through backtest data, and calculating each factor with the data. Including:  
**Market Premium** - Calculated from S&P/TSX Composite Index and 90-day Treasury Bills.  
**SMB** - Categorized each security as small or large market cap by using 30-70 percentiles of aggregate market cap in portfolio.  
**HML** - Used inverse of P/B as proxy to catergorize and calculate book to market value.  

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.api as sm
#from sklearn.linear_model import LinearRegression
import scipy, scipy.stats
pd.options.mode.chained_assignment = None  # default='warn'

C:\Users\Ry\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
#Set years of data to look at: 2007-2018
start = 2007
end = 2018

In [3]:
def dfAppend(strat, s, e):
    """
    Reads data and appends to a central df.
    df    = data frame to append to
    strat = Strategies label {A = Aggressive, C = Conservative}
    start = year to start
    end   = year to end, not inclusive of end date
    """
    df = pd.DataFrame()
    fileRange = np.arange(s, e + 1)     
    
    for i in fileRange:
        if i == s:
            dfAdd = pd.read_csv(strat + str(i) + ".csv", skiprows=range(1,4), sep=',', encoding='iso-8859-1')
        else:
            dfAdd = pd.read_csv(strat + str(i) + ".csv", skiprows=range(1,4), sep=',', encoding='iso-8859-1')
        dfAdd = dfAdd.iloc[:, : -1] #drops last extra column
        dfAdd['Year'] = i
        df = df.append(dfAdd, ignore_index = True)

    return df

In [4]:
def dfResetIndex (df):
    """
    Resets index each time a df is made
    """
    
    return df.reset_index(drop = True)

In [5]:
def calcReturn(df):
    """
    Gives return of individual security from portfolio
    """
   
    return round(df.Return.mean(), 4)

In [6]:
def calcRf (df):
    """
    Returns Rf for FF
    """
    df = dfResetIndex (df)
    return round(df.TB90[1], 4)

In [7]:
def fillMktPrem (df, s, e):
    """
    Fills in MktPrem to DF
    """
    dfNew = df
    dfNew['MKtReturn'] = ""
    dfNew['MktPrem'] = ""
    fileRange = np.arange(s, e)    
    for i in fileRange:
        dfNew['MKtReturn'].loc[dfNew['Year'] == i+1] = (dfNew['TRI'].loc[dfNew['Year'] == i+1].iloc[0] / dfNew['TRI'].loc[dfNew['Year'] == i].iloc[0] - 1)*100
        dfNew['MktPrem'].loc[dfNew['Year'] == i+1] = dfNew['MKtReturn'].loc[dfNew['Year'] == i+1].iloc[0] - dfNew['TB90'].loc[dfNew['Year'] == i+1].iloc[0]
    return dfNew

In [8]:
def calcMktPrem (df):
    """
    Returns Mkt Premium for FF
    """
    df = dfResetIndex(df)
    return round(df.MktPrem[1], 4)

In [9]:
def calcSMB(df):
    """
    Returns SMB for FF
    """
    #Define Quantile
    SQuantile = 0.3
    LQuantile = 0.7
    df["SMB"] = ""
    
    #Assigns stock size based on market cap
    df.SMB[df.MKTCAP <= df.MKTCAP.quantile(SQuantile)] = "SCap"
    df.SMB[(df.MKTCAP > df.MKTCAP.quantile(SQuantile)) & (df.MKTCAP < df.MKTCAP.quantile(LQuantile))] = "MCap"
    df.SMB[df.MKTCAP >= df.MKTCAP.quantile(LQuantile)] = "LCap"
    
    #Calculates average return of stocks in portfolio subset based on size
    SmallCapReturn = df.Return.loc[df["SMB"] == "SCap"].mean()
    LargeCapReturn = df.Return.loc[df["SMB"] == "LCap"].mean()
    
    #Returns SMB based on definition
    SMB = SmallCapReturn - LargeCapReturn
    return round(SMB, 4)


In [10]:
def calcHML (df):
    """
    Returns HML for FF
    Uses inverse of P/B as proxy for Book/Mkt
    """
    #Define Quantile
    SQuantile = 0.3
    LQuantile = 0.7
    df["HML"] = ""
    df["BP"] = df.PB**(-1) #Create Book/MktValue Proxy
    
    #Assigns stock size based on market cap
    df.HML[df.BP <= df.BP.quantile(SQuantile)] = "SValue"
    df.HML[(df.BP > df.BP.quantile(SQuantile)) & (df.BP < df.BP.quantile(LQuantile))] = "MValue"
    df.HML[df.BP >= df.BP.quantile(LQuantile)] = "LValue"
    
    #Calculates average return of stocks in portfolio subset based on size
    SmallValueReturn = df.Return.loc[df["HML"] == "SValue"].mean()
    LargeValueReturn = df.Return.loc[df["HML"] == "LValue"].mean()
    
    #Returns SMB based on definition
    HML = SmallValueReturn - LargeValueReturn
    return round(HML, 4)

In [11]:
def cleanColumns(df):
    """
    Cleans up unnecessary characters
    Cleans up columns, removing the extras
    """
    dfNew = df
   
    try:
        dfNew.columns = dfNew.columns.str.replace(' ','')
    except:
        pass
    
    try:
        dfNew.columns = dfNew.columns.str.replace('/','')
    except:
        pass
    
    dfNew = dfNew.rename(columns={"PCHG12M": "Return"})
    
    dfNew = dfNew[['Symbol',
                'Year',
                'Return',
                'TRI',
                'TB90',
                'MKTCAP',
                'PB'
                ]]
    
    return dfNew   

In [12]:
###Set up Data Frame

#Create empty data frame for the strategies
dfA = pd.DataFrame()
dfC = pd.DataFrame()

#Append the list
dfA = dfAppend("A", start, end) #year 2007-2018
dfC = dfAppend("C", start, end) #year 2008-2018


In [13]:
###Clean up Data Frame and preparing for FF model
#Remove space in columns
dfA = cleanColumns(dfA)
dfC = cleanColumns(dfC)

dfA

,Symbol,Year,Return,TRI,TB90,MKTCAP,PB
0,TCM,2007,394.1667,34542.6016,3.90,2009.5763,4.8272
1,BB,2007,196.5454,34542.6016,3.90,50392.2070,17.3708
2,QUX,2007,52.4324,34542.6016,3.90,914.2384,2.0376
3,TRE,2007,300.0000,34542.6016,3.90,2908.6021,2.6323
4,VT,2007,46.1039,34542.6016,3.90,2296.7551,2.1184
5,LNR,2007,89.3727,34542.6016,3.90,1791.8121,2.0731
6,MDI,2007,102.1609,34542.6016,3.90,1035.5814,4.6532
7,ET,2007,112.4800,34542.6016,3.90,1918.1101,17.0584
8,SW,2007,89.2187,34542.6016,3.90,662.2232,3.3381
9,NTR,2007,158.3199,34542.6016,3.90,29561.9219,6.0249


In [14]:
###Fill in MktPrem
#This part should only be done once
dfA = fillMktPrem (dfA, start, end)
dfC = fillMktPrem (dfC, start, end)

In [15]:
###Continue Cleanup
#Drop First Year
dfA = dfA.loc[dfA['Year'] != start]
dfC = dfC.loc[dfC['Year'] != start]

dfA = dfA.reset_index(drop = True)
dfC = dfC.reset_index(drop = True)
#Convert all inputs used to numeric

dfA.iloc[:, 2:] = dfA.iloc[:, 2:].convert_objects(convert_numeric=True)
dfC.iloc[:, 2:] = dfC.iloc[:, 2:].convert_objects(convert_numeric=True)

C:\Users\Ry\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Ry\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  # This is added back by InteractiveShellApp.init_path()


In [16]:
dfA.head(30)

,Symbol,Year,Return,TRI,TB90,MKTCAP,PB,MKtReturn,MktPrem
0,ATA,2008,39.0400,34992.8008,2.4,671.5371,1.4624,1.303316,-1.096684
1,BIR,2008,219.2500,34992.8008,2.4,1435.1438,2.9372,1.303316,-1.096684
2,IOL,2008,-11.5591,34992.8008,2.4,1136.4976,5.7316,1.303316,-1.096684
3,CR,2008,110.6742,34992.8008,2.4,1067.9100,1.6085,1.303316,-1.096684
4,HPX,2008,28.5444,34992.8008,2.4,922.6240,1.3032,1.303316,-1.096684
5,TOG1,2008,183.8667,34992.8008,2.4,2386.6516,2.1461,1.303316,-1.096684
6,AGU,2008,86.1347,34992.8008,2.4,14185.3096,3.4758,1.303316,-1.096684
7,PMCS,2008,17.8746,34992.8008,2.4,1998.7028,2.5971,1.303316,-1.096684
8,PXE,2008,50.8172,34992.8008,2.4,1178.0902,2.9557,1.303316,-1.096684
9,NVA,2008,13.6364,34992.8008,2.4,1186.5299,1.6271,1.303316,-1.096684


In [17]:
#Create Fama French 3 factor model for Aggressive Strategy
FFA = pd.DataFrame(columns =
                  ["Year",
                   "Return",
                   "Rf",
                   "MktPrem",
                   "SMB",
                   "HML"                    
                  ])
FFAIndex = 0 
for i in range(start+1, end+1):
    FFA.loc[FFAIndex] = [i, 
                    calcReturn(dfA.loc[dfA['Year'] == i]), 
                    calcRf(dfA.loc[dfA['Year'] == i]), 
                    calcMktPrem(dfA.loc[dfA['Year'] == i]), 
                    calcSMB(dfA.loc[dfA['Year'] == i]), 
                    calcHML(dfA.loc[dfA['Year'] == i])
                   ]
    FFAIndex += 1
FFA['Year'] = FFA['Year'].astype(int)

In [18]:
#Create Fama French 3 factor model for Conservative Strategy
FFC = pd.DataFrame(columns =
                  ["Year",
                   "Return",
                   "Rf",
                   "MktPrem",
                   "SMB",
                   "HML"                    
                  ])
FFCIndex = 0 
for i in range(start+1, end+1):
    FFC.loc[FFCIndex] = [i, 
                    calcReturn(dfC.loc[dfC['Year'] == i]), 
                    calcRf(dfC.loc[dfC['Year'] == i]), 
                    calcMktPrem(dfC.loc[dfC['Year'] == i]), 
                    calcSMB(dfC.loc[dfC['Year'] == i]), 
                    calcHML(dfC.loc[dfC['Year'] == i])
                   ]
    FFCIndex += 1
FFC['Year'] = FFC['Year'].astype(int)

In [19]:
FFA

,Year,Return,Rf,MktPrem,SMB,HML
0,2008,36.8584,2.40,-1.0967,18.7416,43.6863
1,2009,12.8186,0.20,-17.5444,12.4716,36.1373
2,2010,68.1621,0.67,10.8205,14.7011,60.3519
3,2011,52.0113,0.89,8.2170,24.8434,61.1273
4,2012,29.9158,1.03,-5.2575,5.4930,46.9944
5,2013,38.0250,0.99,9.0373,13.0665,10.3304
6,2014,54.0386,0.94,26.1894,25.7608,66.1083
7,2015,20.5506,0.37,-9.0489,-17.1839,61.2438
8,2016,74.2083,0.50,8.1886,26.1487,18.0131
9,2017,31.4535,0.71,6.5242,-0.7788,21.5734


In [20]:
FFC

,Year,Return,Rf,MktPrem,SMB,HML
0,2008,-9.2666,2.40,-1.0967,4.2731,21.5956
1,2009,-5.4656,0.20,-17.5444,-1.4298,-2.2787
2,2010,15.8135,0.67,10.8205,11.8667,-8.5265
3,2011,8.0557,0.89,8.2170,13.6852,13.4448
4,2012,5.7244,1.03,-5.2575,4.1007,5.9682
5,2013,20.0063,0.99,9.0373,7.6418,-6.0691
6,2014,17.3592,0.94,26.1894,-3.0535,3.7997
7,2015,-1.1869,0.37,-9.0489,-1.8108,23.3356
8,2016,4.3359,0.50,8.1886,-6.1322,4.7550
9,2017,6.4929,0.71,6.5242,-0.8448,-4.3763


In [21]:
#Set up regression
Y = FFA.Return.values 
X = FFA[["MktPrem","SMB","HML"]]


model = sm.OLS( Y.astype(float), X.astype(float) )

result = model.fit()
print (result.params)
print(result.summary())

MktPrem    0.662782
SMB        0.994160
HML        0.545028
dtype: float64
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.877
Model:                            OLS   Adj. R-squared:                  0.831
Method:                 Least Squares   F-statistic:                     19.08
Date:                Tue, 25 Sep 2018   Prob (F-statistic):           0.000528
Time:                        11:42:09   Log-Likelihood:                -45.925
No. Observations:                  11   AIC:                             97.85
Df Residuals:                       8   BIC:                             99.04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------

C:\Users\Ry\Anaconda3\lib\site-packages\scipy\stats\stats.py:1334: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=11
  "anyway, n=%i" % int(n))


In [22]:
#Set up regression
Y = FFC.Return.values 
X = FFC[["MktPrem","SMB","HML"]]
X = sm.add_constant(X)
X.rename(columns = {"const":"Intercept"}, inplace = True)


model = sm.OLS( Y.astype(float), X.astype(float) )

result = model.fit()
print (result.params)
print(result.summary())


Intercept    5.810956
MktPrem      0.528004
SMB          0.151975
HML         -0.310956
dtype: float64
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.761
Model:                            OLS   Adj. R-squared:                  0.659
Method:                 Least Squares   F-statistic:                     7.436
Date:                Tue, 25 Sep 2018   Prob (F-statistic):             0.0140
Time:                        11:42:09   Log-Likelihood:                -31.609
No. Observations:                  11   AIC:                             71.22
Df Residuals:                       7   BIC:                             72.81
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------

C:\Users\Ry\Anaconda3\lib\site-packages\scipy\stats\stats.py:1334: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=11
  "anyway, n=%i" % int(n))
